In [50]:
from collections import OrderedDict
import gzip
import pandas as pd
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def count_comments(filename):
    """Count comment lines (those that start with "#") in an optionally
    gzipped file.
    :param filename:  An optionally gzipped file.
    """
    #print('in _count_comments')
    comments = 0
    fn_open = gzip.open if filename.endswith('.gz') else open
    with fn_open(filename) as fh:
        for line in fh:
            if line.startswith('#'):
                comments += 1
            else:
                break
    return comments

In [3]:
VCF_HEADER = ['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', '20']


def vcf_to_dataframe(filename, large=True):
    """Open an optionally gzipped VCF file and return a pandas.DataFrame with
    each INFO field included as a column in the dataframe.
    Note: Using large=False with large VCF files. It will be painfully slow.
    :param filename:    An optionally gzipped VCF file.
    :param large:       Use this with large VCF files to skip the ## lines and
                        leave the INFO fields unseparated as a single column.
    """
    if large:
        # Set the proper argument if the file is compressed.
        comp = 'gzip' if filename.endswith('.gz') else None
        # Count how many comment lines should be skipped.
        comments = count_comments(filename)
        # Return a simple DataFrame without splitting the INFO column.
        return pd.read_table(filename, compression=comp, skiprows=comments,
                             names=VCF_HEADER, usecols=range(10))

    # Each column is a list stored as a value in this dict. The keys for this
    # dict are the VCF column names and the keys in the INFO column.
    result = OrderedDict()
    # Parse each line in the VCF file into a dict.
    for i, line in enumerate(lines(filename)):
        for key in line.keys():
            # This key has not been seen yet, so set it to None for all
            # previous lines.
            if key not in result:
                result[key] = [None] * i
        # Ensure this row has some value for each column.
        for key in result.keys():
            result[key].append(line.get(key, None))

    return pd.DataFrame(result)

In [4]:
O15_unfilt = vcf_to_dataframe('vcf/O15_B003776_unfilt.vcf')
O15_unfilt

/Users/lincoln.harris/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.


,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,20
0,chr1,14610,.,T,C,62.74,.,AC=2;AF=1.00;AN=2;DP=11;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,2:2:6:90,6,0"
1,chr1,14640,.,C,T,56.74,.,AC=2;AF=1.00;AN=2;DP=38;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,2:2:6:84,6,0"
2,chr1,629906,rs1578391,C,T,314.77,.,AC=2;AF=1.00;AN=2;DB;DP=253;ExcessHet=3.0103;F...,GT:AD:DP:GQ:PL,"1/1:0,10:10:30:343,30,0"
3,chr1,630026,rs6594029,C,T,60.94,.,AC=2;AF=1.00;AN=2;DB;DP=70;ExcessHet=3.0103;FS...,GT:AD:DP:GQ:PL,"1/1:0,2:2:6:81,6,0"
4,chr1,630161,rs6594031,A,G,56.74,.,AC=2;AF=1.00;AN=2;DB;DP=5;ExcessHet=3.0103;FS=...,GT:AD:DP:GQ:PL,"1/1:0,2:2:6:84,6,0"
5,chr1,630211,rs7416152,C,T,180.90,.,AC=2;AF=1.00;AN=2;DB;DP=44;ExcessHet=3.0103;FS...,GT:AD:DP:GQ:PL,"1/1:0,5:5:15:209,15,0"
6,chr1,630317,rs3021087,A,G,305.78,.,AC=2;AF=1.00;AN=2;DB;DP=72;ExcessHet=3.0103;FS...,GT:AD:DP:GQ:PL,"1/1:0,8:8:24:334,24,0"
7,chr1,630490,.,T,C,127.03,.,AC=2;AF=1.00;AN=2;DP=21;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,4:4:12:155,12,0"
8,chr1,630557,rs7417504,T,C,15.65,.,AC=2;AF=1.00;AN=2;DB;DP=2;ExcessHet=3.0103;FS=...,GT:AD:DP:GQ:PL,"1/1:0,1:1:3:42,3,0"
9,chr1,630596,.,C,T,15.65,.,AC=2;AF=1.00;AN=2;DP=3;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,1:1:3:42,3,0"


In [9]:
set(O15_unfilt['FORMAT'])

{'GT:AD:DP', 'GT:AD:DP:GQ:PL'}

In [15]:
# got it!
for i in range(0, len(O15_unfilt.index)):
    row = O15_unfilt.iloc[i]
    extra_col = row['20']
    AD = extra_col.split(':')[1]
    print(AD)

0,2
0,2
0,10
0,2
0,2
0,5
0,8
0,4
0,1
0,1
0,3
0,1
0,1
0,1
0,1
0,4
0,4
0,4
1,4
0,9
0,59
0,1
0,1
0,16
0,6
0,4
0,2
0,4
0,1
0,2
0,13
0,7
0,4
0,37
81
42
320
0,206
0,101
6,13
0,20
0,12
0,7
2,4
1,5
0,11
0,11
0,2
0,6
0,2
0,9
12,4
0,4
11,3
11,3
0,23
13,11
13,11
14,11
5,7
0,43
35,121
141,20
48,109
156,21
4,5
0,9
0,10
5,5
7,4
0,11
0,6
4,2
1,11
1,11
20,26
34,11
18,6
0,2
0,23
29,7
52,8
34,22
0,13
0,2
0,2
32,12
0,15
12,11
12,11
0,19
66,13
12,5
12,5
12,5
0,5,12
5,12
12,5
0,27
36,8
45,8
45,8
35,7
10,4
10,22
0,18
0,5
0,5
0,0
0,4
0,4
0,13
1,13
1,13
5,2
5,2
5,2
5,2
5,2
1,21
0,4
0,5
0,4
0,14
0,2
0,2
0,2
0,2
0,2
0,2
136,12
0,37
0,29
63,11
39,8
39,8
42,14
0,56
2,3
0,5
0,5
0,10
4,6
0,4
0,4
0,8
0,19
0,12
0,27
0,7
1,4
1,4
1,4
0,10
0,10
0,10
25,3
0,11
13,4
13,4
13,4
13,4
5,6
5,6
10,2
3,9
3,9
7,9
17,9
17,9
0,1
0,1
0,1
0,1
0,2
0,2
3,2
10,10
10,10
10,10
10,10
7,20
19,4
19,4
19,4
4,15
4,15
0,4
0,9
0,12
0,3
0,3
0,2
60,9
0,11
0,57
9,4
0,30
15,8
45,8
33,5
0,7
0,7
0,5
0,2
7,9
0,12
12,3
0,5
0,7
0,7
0,4
0,4
0,12
0,4
1,6
9

26,10
23,10
23,10
22,10
28,4
22,10
22,10
22,10
22,10
22,10
4,31
3,15
16,4
16,4
25,3
24,4
20,8
0,5
0,17
28,12
35,5
27,6
15,4
15,4
15,4
15,4
3,8
41,29
29,4
2,1
2,1
0,2
0,2
0,2
0,2
0,2
0,2
0,2
0,2
0,2
0,2
14,9
14,9
14,9
14,9
14,9
5,9
5,12
5,12
5,12
5,7
8,4
8,4
0,2
0,4
0,4
0,14
0,33
0,6
0,11
0,1
6,4
0,12
6,2
2,4
0,11
0,10
0,10
0,46
0,15
0,38
13,5
0,5
0,10
14,7
16,7
16,7
16,7
16,7
16,7
16,7
16,7
16,7
2,15
2,15
2,15
44,9
0,9
0,9
0,9
0,2
35,15
3,11
0,7
0,21
0,9
0,9
50,7
22,10
22,10
22,10
5,6
0,23
0,23
0,23
0,23
0,23
0,9
19,10
13,35
16,19
15,19
0,11
0,11
0,11
0,11
0,35
12,2
0,4
0,28
0,5
0,7
0,34
4,3
16,8
24,8
0,16
2,9
0,12
0,17
3,6
25,4
34,8
0,24
0,0
0,47
22,10
5,15
11,2
11,2
11,2
0,2
0,13
13,18
3,7
0,3
0,29
38,5
0,3
0,3
0,4
0,8
0,8
0,8
27,4
29,4
0,9
0,72
21,7
35,15
42,7
0,14
0,30
0,37
0,1
0,11
0,2
0,2
37,11
37,11
29,11
0,50
17,4
13,6
13,6
13,6
13,6
13,6
23,17
0,53
0,6
0,18
0,18
0,11
0,11
0,3
0,9
0,17
0,12
0,47
0,5
0,2
0,4
0,4
0,4
0,4
0,4
0,4
9,2
0,46
32,17
0,4
2,2
3,17
0,20
3,11
0,7
10,3
7,3


0,2
5,3
0,6
0,24
0,2
0,4
0,4
0,10
0,6
0,6
3,9
17,3
17,10
0,24
36,5
0,47
0,74
0,70
0,3
1,2
12,13
17,12
17,12
17,12
0,30
0,30
4,2
4,2
0,2
0,2
27,5
24,13
22,5
0,11
0,22
0,22
0,22
0,24
3,21
0,24
0,7
6,10
23,7
0,4
27,7
0,3
0,3
0,12
0,12
78,17
27,11
52,15
52,15
52,15
0,11
0,11
6,9
17,7
17,16
16,17
20,18
3,44
34,8
34,8
0,59
41,5
69,8
77,8
71,8
76,8
0,22
0,22
0,67
35,6
16,6
13,4
0,44
5,4
13,5
0,4
0,4
0,2
0,15
0,47
18,45
45,18
0,86
0,13
0,7
0,6
0,5
0,1
0,2
0,2
0,6
0,14
0,14
0,14
0,7
4,12
0,2
0,2
0,6
0,12
64,13
36,45
54,42
0,20
0,18
119
14,4
0,2
0,2
0,2
4,2
0,3
0,2
56,20
18,13
18,13
15,10
29,6
29,6
29,6
35,6
35,6
21,6
16,5
30,5
30,5
0,39
2,5
2,5
2,5
0,2
0,2
0,2
0,3
3,5
0,4
0,4
0,4
0,2
0,12
0,15
0,29
15,3
15,3
15,7
19,3
19,3
0,7
0,9
0,16
0,37
0,2
0,15
0,8
0,8
0,7
0,7
0,16
40,5
0,4
34,6
0,9
0,24
9,7
9,7
30,5
30,5
1,3
24,4
2,2
4,2
4,2
2,2
2,2
2,2
2,2
2,2
2,2
2,2
9,4
11,2
11,2
9,4
26,5
0,9
7,2
7,2
6,8
12,9
1,12
1,12
1,12
0,5
0,4
0,11
1,29
7,2
1,5
0,3
0,6
11,7
2,7
3,32
16,16
0,3
28,13
5,4
0,5
0,11
2,

8,5
8,5
2,6
0,12
9,16
0,7
0,10
0,4
0,4
1,8
27,12
5,4
0,13
0,13
0,13
0,1
0,1
0,1
0,3
0,3
0,3
0,7
0,10
2,3
13,13
19,6
22,3
4,3
4,3
4,3
22,5
13,9
0,34
0,12,31
0,46
0,5
0,9
0,9
0,9
0,5
0,2
0,4
0,4
0,4
0,4
0,4
0,4
0,4
0,4
0,4
0,4
0,4
0,8
28,4
7,4
7,4
0,1
0,2
0,7
0,7
0,7
0,7
8,4
1,21
0,16
6,15
21,9
0,4
4,2
4,2
4,2
4,10
0,14
18,4
18,4
18,4
18,4
1,6
4,13
0,6
11,2
6,7
16,13
63,7
39,7
51,10
56,10
0,8
64,8
24,4
42,5
0,1
40,14
41,19
0,26
25,7
27,6
0,4
0,10
0,10
0,10
0,10
17,10
17,10
6,3
10,6
0,7
0,11
22,15
0,1
0,1
0,1
0,16
11,5
11,5
11,5
11,5
12,6
0,52
0,109
17,6
0,9
0,34,5
37,5
37,10
39,5
0,53
0,10,5
37,5
0,5
0,5
0,60
0,26
0,3
0,26
0,2
0,2
0,18
0,11
0,11
0,15
0,15
0,8
0,3
3,8
3,8
8,8
8,8
11,5
31,21
31,21
0,2
7,3
16,6
0,3
0,3
1,2
1,2
1,2
1,2
1,2
1,2
1,2
1,2
41,8
0,7
0,2
0,2
0,2
0,5
0,5
0,16
15,10
0,2
0,5
0,9
0,4
0,12
0,40
0,4
0,4
0,23
0,8
0,9
0,2
16,6
0,41
0,5
0,18
0,19
0,8
0,5
0,12
0,12
0,15
0,8
0,30
4,12
0,21
0,14
12,15
18,12
12,12
12,12
12,12
0,4
0,4
0,11
8,3
0,6
0,5
0,5
0,5
0,5
0,5
0,5
0,5
0,5

35,5
10,12
10,12
0,10
0,7
0,2
0,2
0,2
18,3
19,14
0,27
0,10
0,1
0,8
0,51
0,70
0,70
61,10
0,17
0,17
0,5
24,3
24,3
0,2
0,10
0,9
174,34
0,17
0,3
0,10
0,10
0,5
65,10
102,16
87,10
0,17
0,5
0,5
0,4
0,14
41,26
0,25
28,7
0,82
72,15
0,12
0,5
42,10
8,4
6,3
6,3
4,3
0,6
0,12
0,16
0,11
9,7
9,7
9,7
0,28
0,23
0,42
0,10
0,14
0,4
0,6
0,5
0,16
8,8
8,8
0,9
0,4
0,1
0,4
0,1
0,4
0,13
0,4
0,14
0,5
0,42
0,15
0,2
17,15
0,3
0,8
0,26
0,23
0,19
15,6
15,6
6,4
0,5
0,8
0,5
0,3
0,3
0,3
0,5
0,8
0,5
0,24
0,16
0,11
0,11
0,38
0,38
0,38
17,11
61,9
0,71
1,62
43,13
38,17
0,15
0,31
0,25
48,8
11,10
24,9
24,8
0,8
0,2
0,6
1,3
5,15
0,6
0,3
0,2
0,2
0,2
0,19
8,4
0,6
0,11
0,11
63,9
63,9
67,9
67,9
67,2
6,19
6,19
0,5
0,5
0,25
0,1
0,1
0,1
0,1
0,1
0,1
0,2
28,4
0,3
0,5
0,0
0,2
0,2
0,2
0,3
30,7
0,60
11,4
0,5
0,11
75,11
0,77
0,73
0,21
0,8
0,12
0,39
0,39
32,11
19,6
42,10
0,28
0,11
1,10
0,5
1,12
0,6
0,3
0,1
3,3
0,4
0,4
0,2
19,4
0,12
0,12
0,15
16,8
0,8
20,13
20,13
20,13
0,4
0,4
0,4
0,4
36,4
4,13
5,19
23,9
0,2
2,4
0,13
25,4
30,6
0,7
0,4
11,2
5

...

32,4
0,41
0,22
0,22
5,2
5,2
0,4
0,2
0,4
0,2
3,8
10,2
10,2
7,2
5,2
5,2
5,2
5,2
2,2
1,2
0,3
0,2
0,2
3,5
3,5
3,5
3,5
0,2
0,2
0,11
11,11
0,8
0,8
0,19
0,14
12,9
0,9
4,2
4,2
4,2
0,2
23,14
0,9
0,4
0,4
0,28
22,10
0,4
0,31
0,17
4,4
0,11
31,10
4,7
4,7
0,2
0,2
0,10
3,5
0,5
0,5
5,3
0,5
0,3
0,3
0,3
0,3
0,3
5,5
5,5
5,5
0,5
0,5
0,5
0,5
0,5
0,5
8,16
30,6
11,6
0,2
0,2
0,5
17,5
0,20
0,11
0,11
7,8
0,2
0,2
0,2
0,2
0,8
42,7
0,15
0,15
8,6
0,14
0,14
0,13
0,3
0,15
0,3
0,3
0,3
0,6
0,6
47,31
0,17
0,5
0,11
0,3
0,3
0,3
0,3
0,3
0,3
0,3
0,3
0,3
0,3
6,3
0,8
1,8
0,6
0,11
50,16
56,8
0,21
10,4
12,3
0,5
28,30
28,6
0,4
0,4
0,36
0,36
0,37
0,39
0,28
0,31
46,11
0,66
0,58
4,20
3,2
6,5
5,7
0,23
0,23
7,7
0,2
5,2
0,7
0,7
0,7
30,11
0,2
0,3
11,7
0,4
36,10
16,4
16,4
16,4
16,4
16,4
19,4
19,4
17,5
17,5
0,11
10,11
10,11
5,11
15,5
9,11
13,5
2,5,11
13,5
7,11
7,11
13,5
5,11
0,5
0,5
0,5
28,12
0,37
36,16
52,5
89,12
89,11
40,22
58,6
75,19
74,11
15,8
0,13
0,1
2,3
0,17
28,5
0,16
0,2
0,6
0,4
31,8
25,8
0,2
0,2
0,2
0,2
0,2
1,179
154,16
87,10
66

0,2
0,9
0,12
0,3
0,1
2,3
1,6
15,5
5,15
5,15
5,15
52,8
0,8
12,2
0,14
0,2
0,2
0,2
0,2
0,3
0,4
0,4
0,3
40,29
7,7
8,32
0,29
0,5
0,5
0,2
28,11
0,38
0,14
0,25
17,5
18,7
17,10
11,12
19,10
11,4
12,3
12,3
52,13
50,8
150,17
68,9
139,14
71,9
14,13
14,13
0,4
0,21
11,2
11,2
11,2
11,2
0,6
18,6
0,4
0,19
0,7
0,10
4,4
9,3
9,3
0,9
0,6
0,5
0,44
0,43
19,7
30,7
14,6
6,8
6,8
0,29
0,45
1,4
62,11
50,10
0,7
6,7
19,10
0,3
0,5
0,54
71,12
19,3
7,8
13,2
0,12
13,2
13,4
17,4
44,6
44,6
9,4
9,4
9,4
3,4
0,2
0,2
0,3
0,5
8,7
0,2
0,5
0,5
1,1
1,1
1,1
0,10
4,5
0,4
0,11
0,13
0,13
0,11
0,19
0,5
0,2
0,2
16,18
0,26
0,32
0,18
0,6
0,11
0,16
14,9
15,10
2,8
1,24
3,19
7,5
16,4
16,4
16,4
16,4
16,4
16,4
16,4
16,4
16,4
0,4
0,4
0,4
0,1
0,20
2,6
0,9
0,6
0,2
26,18
8,2
0,35
4,9
4,7
4,7
0,4
0,4
0,4
0,3
0,4
0,11
24,16
0,8
56,14
30,24
34,6
0,1
0,1
0,1
6,26
0,2
0,7
2,6
2,6
2,6
0,6
0,6
0,6
0,6
0,6
1,6
1,6
86,25
23,41
0,2
0,4
0,8
0,4
0,9
0,9
0,13
0,27
0,27
0,7
0,25
0,20
30,4
15,7
11,4
11,4
11,4
0,5
0,3
0,3
23,8
16,4
12,6
0,19
0,21
0,5
0,5
0,7
12

25,4
8,9
42,6
42,6
33,6
33,6
1,13
0,10
18,7
28,23
0,8
0,8
0,5
0,13
0,13
0,10
25,10
0,2
0,2
0,9
0,11
0,6
37,5
0,3
25,12
22,5
24,5
18,4
0,7
0,10
2,3
14,3
0,10
1,6
6,4
6,4
20,9
0,16
0,16
0,1
0,5
0,2
0,2
0,9
0,7
0,3
0,3
0,3
0,3
0,3
77,8
43,41
0,104
0,131
82,8
8,2
8,2
4,4
4,4
4,4
5,2
0,10
11,4
0,8
0,23
0,11
37,5
0,46
6,10
0,9
0,7
0,7
0,7
24,20
7,3
9,11
29,16
0,2
0,30
0,25
0,7
0,5
0,5
0,5
0,7
0,7
0,7
0,3
0,39
0,15
0,15
9,7
11,3
11,3
11,3
11,3
11,3
11,3
0,7
0,8
11,5
14,2
37,20
35,21
0,8
0,13
3,5
9,3
16,9
16,9
0,6
4,6
0,9
55,8
98,11
98,11
101,11
58,27
8,33
0,6
2,4
0,12
0,8
0,7
0,6
0,3
17,3
17,3
0,34
0,34
0,18
0,23
6,11
7,3
19,5
49,9
10,3
0,2
0,2
0,2
0,1
21,8
0,2
10,3
4,2
4,2
0,12
19,12
23,5
31,4
0,65
0,36
1,1
4,2
0,21
0,19
20,3
0,11
6,2
6,2
31,5
0,22
2,12
1,2
0,3
1,13
0,24
0,16
16,5
0,9
2,8
0,7
0,5
0,15
11,16
11,16
0,10
11,14
0,29
62,7
62,7
62,7
62,7
63,7
40,12
70,13
80,10
37,15
6,3
0,7
0,5
0,3
21,12
6,31
6,28
0,10
5,5
0,5
0,5
0,2
0,23
30,7
40,8
0,88
97,21
97,21
79,16
5,10
5,10
2,2
0,9
0,2
0,7

0,3
22,8
0,7
0,7
46,9
0,46
0,3
0,22
0,14
0,14
1,30
0,34
0,2
0,2
0,2
0,2
0,7
0,2
0,2
0,2
1,2
51,18
0,6
0,30
0,26
3,5
5,3
3,7
4,6
17,30
17,30
39,9
0,14
0,6
0,4
0,10
2,70
0,11
16,4
0,8
1,16
11,5
0,35
21,6
0,6
0,2
0,2
0,130
0,17
0,3
0,6
0,6
0,8
0,7
0,8
0,2
0,2
0,1
0,2
0,2
0,67
121,16
0,64
53,11
6,7
6,7
6,7
23,5
22,4
0,6
0,3
0,3
8,2
0,9
0,9
0,9
0,5
0,5
0,10
0,2
4,2
0,1
0,3
0,2
0,4
0,2
0,14
0,4
0,75
65,9
0,20
0,1
10,9
26,3
14,6
0,1
2,2
47,13
56,13
17,16
0,18
2,8
30,10
44,6
0,1
0,1
0,2
0,2
0,24
0,5
0,13
0,54
0,3
6,2
7,2
7,2
7,2
0,6
0,1
0,1
0,1
0,29
32,5
32,5
22,4
22,4
0,26
5,4
5,4
0,28
0,52
72,10
55,6
79,23
12,52
6,4
0,3
7,3
7,3
7,3
7,3
0,27
0,9
17,7
15,6
0,6
0,32
31,6
20,7
4,7
8,3
1,10
7,2
7,2
7,2
15,2
12,2
28,4
61,10
54,11
79,19
24,9
38,38
48,27
13,2
0,5
0,10
0,10
0,26
25,16
28,13
18,3
10,6
10,6
10,6
7,8
8,6
10,0
5,3
4,3
5,3
5,3
6,3
6,3
12,2
9,2
7,2
7,2
0,2
0,2
0,2
7,3
7,3
15,9
0,2
0,2
0,2
0,2
0,39
14,10
14,4
17,12
0,7
0,8
0,8
0,16
0,14
0,8
0,11
0,11
0,3
0,3
12,9
30,5
0,42
0,4
0,4
0,4
0,14


KeyboardInterrupt: 

In [68]:
def ROI_df_subset(vcf, chrom, start, end):
    """ subset a single vcf based on genomic coords"""
    chrStr = 'chr' + str(chrom_)
    
    keep0 = vcf['CHROM'] == chrStr
    vcf_sub0 = vcf[keep0]

    keep1 = vcf_sub0['POS'] >= start
    vcf_sub1 = vcf_sub0[keep1]

    keep2 = vcf_sub1['POS'] <= end
    vcf_sub2 = vcf_sub1[keep2]
    
    return(vcf_sub2)

In [69]:
def coverage_search(df):
    """ given single vcf entry, search for AD (DepthPerAlleleBySample) col """ 
    for i in range(0, len(df.index)):
        row = df.iloc[i]
        extra_col = row['20']
        AD = extra_col.split(':')[1]
        print(AD)

In [70]:
def ROI_search(row):
    """ search for given ROI, across all cells """
    chrom_ = row['chrom']
    start_ = row['start_pos']
    end_pos = row['end_pos']
    
    for f in cell_files_list:
        vcf_ = vcf_to_dataframe(f)
        vcf_sub = ROI_df_subset(vcf_, chrom_, start_, end_)
    
        if not vcf_sub.empty:
            coverage_search(vcf_sub)

In [81]:
""" main. search for each ROI. """
global cell_files_list

currPATH = os.getcwd()
cells_files_list = os.listdir('vcf/')
cells_files_list = [currPATH + '/vcf/' + s for s in cells_files_list] # list comprehension

ROI_df = pd.read_csv('/Users/lincoln.harris/code/cerebra/py_notebooks/coverageBatch_v1.csv')
ROI_df.apply(ROI_search, axis=1)

NameError: ("name 'cell_files_list' is not defined", 'occurred at index 0')

In [80]:
currPATH = os.getcwd()
cells_files_list = os.listdir('vcf/')
cells_files_list = [currPATH + '/vcf/' + s for s in cells_files_list] # list comprehension
print(cells_files_list)

['/Users/lincoln.harris/code/SNP_calling_pipeline/coverage/vcf/A11_1001000408.vcf', '/Users/lincoln.harris/code/SNP_calling_pipeline/coverage/vcf/A7_B001545.vcf', '/Users/lincoln.harris/code/SNP_calling_pipeline/coverage/vcf/A22_B003771.vcf', '/Users/lincoln.harris/code/SNP_calling_pipeline/coverage/vcf/A22_1001000407.vcf', '/Users/lincoln.harris/code/SNP_calling_pipeline/coverage/vcf/A21_B003528.vcf', '/Users/lincoln.harris/code/SNP_calling_pipeline/coverage/vcf/A22_B003995.vcf', '/Users/lincoln.harris/code/SNP_calling_pipeline/coverage/vcf/A7_B003187.vcf', '/Users/lincoln.harris/code/SNP_calling_pipeline/coverage/vcf/A10_B001007.vcf', '/Users/lincoln.harris/code/SNP_calling_pipeline/coverage/vcf/A13_B003529.vcf', '/Users/lincoln.harris/code/SNP_calling_pipeline/coverage/vcf/A8_B003125.vcf', '/Users/lincoln.harris/code/SNP_calling_pipeline/coverage/vcf/A3_B003067.vcf', '/Users/lincoln.harris/code/SNP_calling_pipeline/coverage/vcf/A1_B003132.vcf', '/Users/lincoln.harris/code/SNP_callin